# Prerequisites

In [75]:
import findspark

# start findspark so notebook can interact with spark
findspark.init()


In [76]:
# what does findspark do? use the ?? magic command to find out
# Note 1: in colab, this may open in a side panel
# Note 2: this magic command is often helpful when encountering an object in a
# notebook that is unfamiliar. More information will be displayed if it exists
?? findspark

Type:        module
String form: <module 'findspark' from '/opt/anaconda3/lib/python3.12/site-packages/findspark.py'>
File:        /opt/anaconda3/lib/python3.12/site-packages/findspark.py
Source:     
"""Find spark home, and initialize by adding pyspark to sys.path.

If SPARK_HOME is defined, it will be used to put pyspark on sys.path.
Otherwise, common locations for spark will be searched.
"""

from glob import glob
import os
import sys

__version__ = "2.0.1"


def find():
    """Find a local spark installation.

    Will first check the SPARK_HOME env variable, and otherwise
    search common installation locations, e.g. from homebrew
    """
    spark_home = os.environ.get("SPARK_HOME", None)

    if not spark_home:
        if "pyspark" in sys.modules:
            return os.path.dirname(sys.modules["pyspark"].__file__)
        for path in [
            "/usr/local/opt/apache-spark/libexec",  # macOS Homebrew
            "/usr/lib/spark/",  # AWS Amazon EMR
            "/usr/local/sp

# 1. Word Count

Instructions:  
For each cell marked "double-click and add explanation here" please answer the question in your own words.  
In the section where you complete the code to perform basic nlp text cleaning and exploration tasks, the goal is to chain all of the transformations together in a single function. For learning and exploration purposes, it is acceptable to have each step seperate, but the last cell in this section should be one function with all transformations chained together.  
For steps c and f, it is acceptable to use your favorite chatbot to generate a list of common stop words (c) and punctuation (e) for use in the code. As these are common steps in nlp/text processing tasks, there are pleanty of libraries to help with this such as nltk, but there is no need to import extra dependencies for this lab unless you are already familiar with working with them.

In [77]:
# start a spark session and create spark context for making rdd
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("word_count") \
    .getOrCreate()

sc = spark.sparkContext

In [78]:
# Defind the rdd
# Data is read line by line
rdd = sc.textFile('../data/data.txt')

In [79]:
# view the first 20 lines of the rdd
rdd.take(20)

['Around the World in Eighty Days',
 '',
 'by Jules Verne',
 '',
 '',
 'Contents',
 '',
 ' CHAPTER I. IN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER, THE ONE AS MASTER, THE OTHER AS MAN',
 ' CHAPTER II. IN WHICH PASSEPARTOUT IS CONVINCED THAT HE HAS AT LAST FOUND HIS IDEAL',
 ' CHAPTER III. IN WHICH A CONVERSATION TAKES PLACE WHICH SEEMS LIKELY TO COST PHILEAS FOGG DEAR',
 ' CHAPTER IV. IN WHICH PHILEAS FOGG ASTOUNDS PASSEPARTOUT, HIS SERVANT',
 ' CHAPTER V. IN WHICH A NEW SPECIES OF FUNDS, UNKNOWN TO THE MONEYED MEN, APPEARS ON ’CHANGE',
 ' CHAPTER VI. IN WHICH FIX, THE DETECTIVE, BETRAYS A VERY NATURAL IMPATIENCE',
 ' CHAPTER VII. WHICH ONCE MORE DEMONSTRATES THE USELESSNESS OF PASSPORTS AS AIDS TO DETECTIVES',
 ' CHAPTER VIII. IN WHICH PASSEPARTOUT TALKS RATHER MORE, PERHAPS, THAN IS PRUDENT',
 ' CHAPTER IX. IN WHICH THE RED SEA AND THE INDIAN OCEAN PROVE PROPITIOUS TO THE DESIGNS OF PHILEAS FOGG',
 ' CHAPTER X. IN WHICH PASSEPARTOUT IS ONLY TOO GLAD TO GET OFF WITH THE LO

In [80]:
# example lambda function
words = rdd.flatMap(lambda lines: lines.split(' '))

In [81]:
# Note and explain the output of the below command
words

PythonRDD[288] at RDD at PythonRDD.scala:53

We are using flatMap to map the result into multiples outputs instead of a single output as the function map does
Each line will be divided into word and the variable "words" will contains all of them, it is a new RDD

<ADD EXPLANATION HERE>

In [107]:
# Note and explain the output of the following command, focusing on the difference with the
# above command
words.collect()[:50] #50 to limit output in terminal

[('Around', 1),
 ('the', 1),
 ('World', 1),
 ('in', 1),
 ('Eighty', 1),
 ('Days', 1),
 ('', 1),
 ('by', 1),
 ('Jules', 1),
 ('Verne', 1),
 ('', 1),
 ('', 1),
 ('Contents', 1),
 ('', 1),
 ('', 1),
 ('CHAPTER', 1),
 ('I.', 1),
 ('IN', 1),
 ('WHICH', 1),
 ('PHILEAS', 1),
 ('FOGG', 1),
 ('AND', 1),
 ('PASSEPARTOUT', 1),
 ('ACCEPT', 1),
 ('EACH', 1),
 ('OTHER,', 1),
 ('THE', 1),
 ('ONE', 1),
 ('AS', 1),
 ('MASTER,', 1),
 ('THE', 1),
 ('OTHER', 1),
 ('AS', 1),
 ('MAN', 1),
 ('', 1),
 ('CHAPTER', 1),
 ('II.', 1),
 ('IN', 1),
 ('WHICH', 1),
 ('PASSEPARTOUT', 1),
 ('IS', 1),
 ('CONVINCED', 1),
 ('THAT', 1),
 ('HE', 1),
 ('HAS', 1),
 ('AT', 1),
 ('LAST', 1),
 ('FOUND', 1),
 ('HIS', 1),
 ('IDEAL', 1)]

The function collect() is used to turn a RDD into an array of object Row(). This function is considered as an action, it asks all the executors to gather data and bring them to the driver

In [83]:
# Use cell magic command to help understand what the rdd.flatMap function is doing in the next cell.
# Insert a text/markdown cell and explain in your own words.

In [84]:
# Initialize a word counter by creating a tuple with word and cound of 1
words = rdd.flatMap(lambda lines: lines.split(' ')) \
                    .map(lambda word: (word, 1))

#remove the print statement to avoid flooding the output
#for w in words.collect(): --- IGNORE ---

In [85]:
# a. count the occurence of each word
# We start by mapping the line to get a tuple with (word, 1) to easily count the occurence by making a sum in the following step
# We used reduceByKey to aggregate our data, it take in parameter a lambda function with key, value as inputs

word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

In [86]:
word_counts.take(20)

[(('the', 1), 4306),
 (('World', 1), 1),
 (('in', 1), 987),
 (('Eighty', 1), 1),
 (('Days', 1), 1),
 (('Jules', 1), 1),
 (('Verne', 1), 1),
 (('Contents', 1), 1),
 (('CHAPTER', 1), 74),
 (('WHICH', 1), 76),
 (('PHILEAS', 1), 36),
 (('AND', 1), 17),
 (('PASSEPARTOUT', 1), 20),
 (('ACCEPT', 1), 2),
 (('OTHER,', 1), 2),
 (('THE', 1), 62),
 (('ONE', 1), 2),
 (('AS', 1), 6),
 (('IS', 1), 16),
 (('CONVINCED', 1), 2)]

In [87]:
# b. a common first step in text analysis, change all capital letters to lower case
# In the same way as before, we use a lambda function to handle each word and apply the lower casing operation

words_lower = rdd.flatMap(lambda lines: lines.split(' ')).map(lambda w : w.lower())
words_lower.take(10)

['around', 'the', 'world', 'in', 'eighty', 'days', '', 'by', 'jules', 'verne']

In [88]:
# c. eliminate the stop words.
# For this task, we use the nltk library to get a list of stop words and then filter them out of the previous rdd
# We can see 'the' disappeared from the output

import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

words_no_stopw = words_lower.filter(lambda w: w not in stop_words)
words_no_stopw.take(10)

['around', 'world', 'eighty', 'days', '', 'jules', 'verne', '', '', 'contents']

In [89]:
# d. sort in alphabetical order
# Same here, we use the sortBy function and apply the lambda function to each word
# We add a filter to remove empty words which could appear in first position

words_ordered = words_no_stopw.sortBy(lambda w: w).filter(lambda w: w != '')
words_ordered.take(10)


['$5,000)',
 '&c.,',
 '($1',
 '(801)',
 '(a)',
 '(and',
 '(any',
 '(b)',
 '(c)',
 '(does']

In [90]:
# e. sort descending by word frequency

words_desc = word_counts.sortBy(lambda x: x[1], ascending=False)
words_desc.take(10)

[(('the', 1), 4306),
 (('', 1), 2149),
 (('of', 1), 1870),
 (('and', 1), 1790),
 (('to', 1), 1689),
 (('a', 1), 1261),
 (('in', 1), 987),
 (('was', 1), 974),
 (('his', 1), 804),
 (('he', 1), 720)]

In [91]:
import re

def remove_punctuation(w) :
    return re.sub(r'[^\w\s]', '', w)

words_no_punct = words_lower.map(remove_punctuation)
words_no_punct.take(10)

['around', 'the', 'world', 'in', 'eighty', 'days', '', 'by', 'jules', 'verne']

# 2. What does the following cell block do?
Comment the code below line by line after the provided hash-tag. You should be able to explain each line while respecting the pep8 style guide of 79 characters or less per line!

In [92]:
 # Create an RDD of tuples (name, age)
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25)]) # converting a list into a RDD

# Try to undestand what this code does (line by line)
agesRDD = (dataRDD
  .map(lambda x: (x[0], (x[1], 1))) # creat a new tuple based on the previous one with this schema : (name, (age, 1))
  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) # Aggregate by taking the name as the key and the value as the tuple (age, 1)
  # for each, sum of ages and counter
  .map(lambda x: (x[0], x[1][0]/x[1][1]))) # mapping again by creating tuple (name, age/count)

In [93]:
agesRDD.take(10)

[('Brooke', 22.5), ('Denny', 31.0), ('TD', 35.0), ('Jules', 30.0)]

# Where to go from here

Further exploration for students who complete the lab before the end of the session or want to go further.

- perform eda on the original french version of the [book](https://www.gutenberg.org/ebooks/46541.txt.utf-8) and compare the two
- recomplete the exercises using a the docker install
- install java and spark directly onto host machine and either rexplore this notebook or perform eda on other data sets
- write a simple python timer function for seeing how quickly your rdd runs as written. change the order of the steps in order to make the rdd run as optimally as possible

In [94]:
# Get French data for labs - corrected URL and path
!wget -nc -O le_tour_du_monde_en_quatre_vingts_jours.txt https://www.gutenberg.org/cache/epub/46541/pg46541.txt

--2025-09-18 15:21:23--  https://www.gutenberg.org/cache/epub/46541/pg46541.txt
Résolution de www.gutenberg.org (www.gutenberg.org)… 2610:28:3090:3000:0:bad:cafe:47, 152.19.134.47
Connexion à www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 472772 (462K) [text/plain]
Sauvegarde en : « le_tour_du_monde_en_quatre_vingts_jours.txt »

le_tour_du_monde_en 100%[===================>] 461,69K  1,25MB/s    ds 0,4s    

2025-09-18 15:21:23 (1,25 MB/s) — « le_tour_du_monde_en_quatre_vingts_jours.txt » sauvegardé [472772/472772]



### 1. French Version Analysis - Le Tour du Monde en Quatre-Vingts Jours

Following the same methodology as the English version, we will now analyze the original French text of "Around the World in Eighty Days" by Jules Verne.

In [95]:
# View the first lines of the French RDD
rdd_french = sc.textFile('le_tour_du_monde_en_quatre_vingts_jours.txt')
rdd_french.take(20)

['The Project Gutenberg eBook of Le Tour du monde en quatre-vingts jours',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: Le Tour du monde en quatre-vingts jours',
 '',
 'Author: Jules Verne',
 '',
 'Illustrator: Léon Benett',
 '        Alphonse Marie de Neuville',
 '',
 'Release date: August 9, 2014 [eBook #46541]',
 '                Most recently updated: October 24, 2024',
 '']

In [96]:
# Initialize French word counter by creating tuples with word and count of 1
words_french = rdd_french.flatMap(lambda lines: lines.split(' ')) \
                         .map(lambda word: (word, 1))

# Print first 20 French words with their counts
for w in words_french.take(20):
    print(w)

('The', 1)
('Project', 1)
('Gutenberg', 1)
('eBook', 1)
('of', 1)
('Le', 1)
('Tour', 1)
('du', 1)
('monde', 1)
('en', 1)
('quatre-vingts', 1)
('jours', 1)
('', 1)
('', 1)
('', 1)
('', 1)
('', 1)
('This', 1)
('ebook', 1)
('is', 1)


In [97]:
# a. Count French words with reduceByKey
word_counts_french = words_french.reduceByKey(lambda a, b: a + b)
word_counts_french.take(10)

[('Gutenberg', 22),
 ('eBook', 4),
 ('of', 112),
 ('Tour', 3),
 ('du', 763),
 ('quatre-vingts', 23),
 ('', 5389),
 ('for', 22),
 ('use', 9),
 ('anyone', 4)]

In [98]:
# b. Convert French words to lowercase
words_french_lower = rdd_french.flatMap(lambda lines: lines.split(' ')).map(lambda w: w.lower())
words_french_lower.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'le',
 'tour',
 'du',
 'monde',
 'en']

In [99]:
# Sort French word counts by frequency descending
word_counts_french_sorted = word_counts_french.sortBy(lambda x: x[1], ascending=False)
word_counts_french_sorted.take(10)

[('', 5389),
 ('de', 2851),
 ('le', 1609),
 ('à', 1597),
 ('la', 1501),
 ('et', 1371),
 ('les', 958),
 ('du', 763),
 ('en', 707),
 ('il', 684)]

In [100]:
# c. Eliminate French stop words
# Using nltk library to get French stop words and filter them out
import nltk
from nltk.corpus import stopwords

# Download French stopwords if not already available
nltk.download('stopwords', quiet=True)
french_stop_words = set(stopwords.words('french'))

words_french_no_stopw = words_french_lower.filter(lambda w: w not in french_stop_words)
words_french_no_stopw.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'tour',
 'monde',
 'quatre-vingts',
 'jours',
 '']

In [101]:
# d. Sort French words in alphabetical order
# Filter empty words and sort alphabetically
words_french_ordered = words_french_no_stopw.filter(lambda w: w != '').sortBy(lambda w: w)
words_french_ordered.take(10)

['#46541]',
 '$5,000)',
 '($1',
 '(1',
 '(1,875',
 '(10,000',
 '(100,000',
 '(100,000',
 '(12,500',
 '(125,000']

In [102]:
# e. Count and sort French words by frequency
word_counts_french_clean = words_french_no_stopw.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts_french_clean_sorted = word_counts_french_clean.sortBy(lambda x: x[1], ascending=False)
word_counts_french_clean_sorted.take(10)

[('', 5389),
 ('fogg', 367),
 ('phileas', 328),
 ('plus', 320),
 ('mr.', 286),
 ('cette', 277),
 ('a', 255),
 ('passepartout', 252),
 ('répondit', 214),
 ("qu'il", 194)]

In [103]:
# f. Remove punctuation and blank spaces from French words
# str.translate removes all punctuation from each word in the RDD
import string
words_french_clean = words_french_no_stopw.map(lambda w: w.translate(str.maketrans('', '', string.punctuation)))
words_french_clean = words_french_clean.filter(lambda w: w.strip() != '')
words_french_clean.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'tour',
 'monde',
 'quatrevingts',
 'jours',
 'this']

### 2. Comparative EDA - French vs English

As requested in "Where to go from here", we will now perform exploratory data analysis comparing the French and English versions of the book.

In [104]:
# Comparative analysis between French and English versions
print("=== COMPARATIVE ANALYSIS: FRENCH vs ENGLISH ===")

# Use results already calculated in previous cells
# For French: use word_counts_french_clean_sorted (from step e in French analysis)
# For English: first ensure we have the English results by recreating if needed

# Create English word count results using the same steps as French
english_word_counts_clean = words_no_stopw.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
english_word_counts_sorted = english_word_counts_clean.sortBy(lambda x: x[1], ascending=False)

print("\n MOST FREQUENT WORDS:")
print("\nFRENCH:")
french_top = word_counts_french_clean_sorted.take(10)
for i, (word, count) in enumerate(french_top, 1):
    print(f"{i:2d}. {word:<15} : {count:>4}")

print("\nENGLISH:")
english_top = english_word_counts_sorted.take(10)
for i, (word, count) in enumerate(english_top, 1):
    print(f"{i:2d}. {word:<15} : {count:>4}")

# Statistical comparison using existing RDD variables
french_unique = word_counts_french_clean_sorted.count()
english_unique = english_word_counts_sorted.count()
french_total = rdd_french.flatMap(lambda line: line.split(' ')).count()
english_total = rdd.flatMap(lambda line: line.split(' ')).count()

print(f"\n STATISTICAL COMPARISON:")
print(f"{'Metric':<25} {'French':<10} {'English':<10} ")
print(f"{'-'*25} {'-'*10} {'-'*10} ")
print(f"{'Total words':<25} {french_total:<10} {english_total:<10} ")
print(f"{'Unique words':<25} {french_unique:<10} {english_unique:<10} ")

=== COMPARATIVE ANALYSIS: FRENCH vs ENGLISH ===

 MOST FREQUENT WORDS:

FRENCH:
 1.                 : 5389
 2. fogg            :  367
 3. phileas         :  328
 4. plus            :  320
 5. mr.             :  286
 6. cette           :  277
 7. a               :  255
 8. passepartout    :  252
 9. répondit        :  214
10. qu'il           :  194

ENGLISH:
 1.                 : 2149
 2. mr.             :  373
 3. fogg            :  365
 4. would           :  274
 5. phileas         :  250
 6. passepartout    :  239
 7. said            :  157
 8. could           :  139
 9. one             :  133
10. fogg,           :  132

 STATISTICAL COMPARISON:
Metric                    French     English    
------------------------- ---------- ---------- 
Total words               76519      68391      
Unique words              15066      11740      


### 3. Performance Optimization and Timer

As requested, we implement timing functions and test different operation orders for optimal performance.

In [105]:
#  Python timer function for measuring RDD performance
import time

def timer(operation_name, rdd_operation):
    # timer function to measure RDD execution time
    print(f"Testing: {operation_name}")
    start = time.time()
    result = rdd_operation.take(10)  # Execute the RDD operation
    end = time.time()
    execution_time = end - start
    print(f"Completed in {execution_time:.3f} seconds")
    return result, execution_time

# Test current word count RDD
print("TESTING RDD PERFORMANCE")
result1, time1 = timer("Current word_counts", words_desc)
print(f"Top words: {result1[:3]}")  # Show first 3 results

TESTING RDD PERFORMANCE
Testing: Current word_counts
Completed in 0.041 seconds
Top words: [(('the', 1), 4306), (('', 1), 2149), (('of', 1), 1870)]


In [106]:
import string

print("OPTIMIZING OPERATION ORDER")

# Approach 1: Original order (a,b,c,d,e,f)
start = time.time()
# a. Initialize word counter with tuples (word, 1)
words_step_a = rdd.flatMap(lambda lines: lines.split(' ')).map(lambda word: (word, 1))
# b. Convert to lowercase 
words_step_b = rdd.flatMap(lambda lines: lines.split(' ')).map(lambda w: w.lower())
# c. Eliminate stop words
words_step_c = words_step_b.filter(lambda w: w not in stop_words)
# d. Sort alphabetically and filter empty words
words_step_d = words_step_c.filter(lambda w: w != '').sortBy(lambda w: w)
# e. Count and sort by frequency
word_counts_step_e = words_step_c.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
result1 = word_counts_step_e.sortBy(lambda x: x[1], ascending=False).take(10)
# f. Remove punctuation (final step)
words_step_f = words_step_c.map(lambda w: w.translate(str.maketrans('', '', string.punctuation)))
words_clean_final = words_step_f.filter(lambda w: w.strip() != '')
time1 = time.time() - start

# Approach 2: Early filtering (b,c,f,a,e,d)
start = time.time()
# b. Convert to lowercase first
words_early_b = rdd.flatMap(lambda lines: lines.split(' ')).map(lambda w: w.lower())
# c. Eliminate stop words early
words_early_c = words_early_b.filter(lambda w: w not in stop_words)
# f. Remove punctuation early
words_early_f = words_early_c.map(lambda w: w.translate(str.maketrans('', '', string.punctuation)))
words_early_f = words_early_f.filter(lambda w: w.strip() != '')
# a. Count words with cleaned data
words_early_a = words_early_f.map(lambda word: (word, 1))
# e. Sort by frequency
word_counts_early_e = words_early_a.reduceByKey(lambda a, b: a + b)
result2 = word_counts_early_e.sortBy(lambda x: x[1], ascending=False).take(10)
# d. Alphabetical sort (optional verification)
words_early_d = words_early_f.sortBy(lambda w: w)
time2 = time.time() - start

# Approach 3: Combined filtering (b,c+f,a,e,d)
start = time.time()
# b. Convert to lowercase
words_combined_b = rdd.flatMap(lambda lines: lines.split(' ')).map(lambda w: w.lower())
# c+f. Combined stop words and punctuation filtering
words_combined_cf = (words_combined_b
                    .map(lambda w: w.translate(str.maketrans('', '', string.punctuation)))
                    .filter(lambda w: w.strip() != '' and w not in stop_words))
# a. Count words
words_combined_a = words_combined_cf.map(lambda word: (word, 1))
# e. Sort by frequency
word_counts_combined_e = words_combined_a.reduceByKey(lambda a, b: a + b)
result3 = word_counts_combined_e.sortBy(lambda x: x[1], ascending=False).take(10)
# d. Alphabetical sort (verification)
words_combined_d = words_combined_cf.sortBy(lambda w: w)
time3 = time.time() - start

# Performance comparison
print("PERFORMANCE RESULTS:")
print(f"Original order (a,b,c,d,e,f):  {time1:.3f} seconds")
print(f"Early filtering (b,c,f,a,e,d): {time2:.3f} seconds") 
print(f"Combined filtering (b,c+f,a,e): {time3:.3f} seconds")

# Find fastest approach
approaches = [("Original order", time1), ("Early filtering", time2), ("Combined filtering", time3)]
fastest = min(approaches, key=lambda x: x[1])
print(f"\nFastest approach: {fastest[0]} ({fastest[1]:.3f} seconds)")


OPTIMIZING OPERATION ORDER
PERFORMANCE RESULTS:
Original order (a,b,c,d,e,f):  0.359 seconds
Early filtering (b,c,f,a,e,d): 0.406 seconds
Combined filtering (b,c+f,a,e): 0.495 seconds

Fastest approach: Original order (0.359 seconds)
